In [1]:
import pandas as pd

In [13]:
# Load the preprocessed ESG and Holdings data
DATA_PATH = "../datasets/"
fund2024 =pd.read_parquet(DATA_PATH +'fund_esg_2024.parquet')

In [14]:
fund2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8645 entries, 0 to 8644
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Industry          8645 non-null   object 
 1   Region            8645 non-null   object 
 2   Country           8645 non-null   object 
 3   Name              8645 non-null   object 
 4   Market_Value_USD  8645 non-null   int64  
 5   Voting            8645 non-null   float64
 6   Ownership         8645 non-null   float64
 7   Portfolio_Weight  8645 non-null   float64
 8   Environmental     8645 non-null   int32  
 9   Social            8645 non-null   int32  
 10  Governance        8645 non-null   int32  
 11  Climate_change    8645 non-null   int32  
 12  ESG_any           8645 non-null   int32  
dtypes: float64(3), int32(5), int64(1), object(4)
memory usage: 709.3+ KB


In [15]:
fund2024.sample(2)

,Industry,Region,Country,Name,Market_Value_USD,Voting,Ownership,Portfolio_Weight,Environmental,Social,Governance,Climate_change,ESG_any
1298,Consumer Discretionary,Europe,Germany,jost werke se,24264354,3.46,3.46,0.001887,0,0,0,0,0
4906,Industrials,North America,United States,beacon roofing supply inc,94573418,1.50,1.50,0.007355,0,0,0,0,0


In [11]:
pip install pandas numpy requests beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# At top of notebook
DATA_PATH = "../datasets/"
TEMP_PATH = "./temp/"
VOTING_DATA_PATH = "../datasets/voting_data/"

# Load data
df_p80 = pd.read_csv(DATA_PATH + 'p80_companies_sorted_with_status.csv')

# Checkpoints will save to temp/
checkpoint_file = TEMP_PATH + 'checkpoint_progress.csv'
disagreements_file = TEMP_PATH + 'checkpoint_disagreements.csv'
stats_file = TEMP_PATH + 'checkpoint_stats.csv'

In [17]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime


# STEP 1: CREATE SAMPLE (P80 + ALL ESG_any=1)

# Calculate P80
p80_threshold = fund2024['Market_Value_USD'].quantile(0.80)

print(f"P80 Market Value threshold: ${p80_threshold:,.0f}")

# Create sample: P80 OR ESG_any=1 (union of both groups)
df_p80 = fund2024[
    (fund2024['Market_Value_USD'] >= p80_threshold) | 
    (fund2024['ESG_any'] == 1)
].copy()

print(f"\nFINAL SAMPLE:")
print(f"Total companies: {len(df_p80)}")
print(f"\nBreakdown:")
print(f"  - P80 only (no ESG_any): {((fund2024['Market_Value_USD'] >= p80_threshold) & (fund2024['ESG_any'] == 0)).sum()}")
print(f"  - ESG_any only (outside P80): {((fund2024['Market_Value_USD'] < p80_threshold) & (fund2024['ESG_any'] == 1)).sum()}")
print(f"  - Both conditions (P80 AND ESG_any): {((fund2024['Market_Value_USD'] >= p80_threshold) & (fund2024['ESG_any'] == 1)).sum()}")

# Verification: all ESG_any=1 must be included
total_esg_any = (fund2024['ESG_any'] == 1).sum()
esg_any_in_sample = (df_p80['ESG_any'] == 1).sum()
print(f"\nESG_any verification:")
print(f"  - Total ESG_any=1 in fund2024: {total_esg_any}")
print(f"  - ESG_any=1 in sample: {esg_any_in_sample}")
print(f"  - ✓ All included: {total_esg_any == esg_any_in_sample}")


P80 Market Value threshold: $81,794,677

FINAL SAMPLE:
Total companies: 1925

Breakdown:
  - P80 only (no ESG_any): 1024
  - ESG_any only (outside P80): 196
  - Both conditions (P80 AND ESG_any): 705

ESG_any verification:
  - Total ESG_any=1 in fund2024: 901
  - ESG_any=1 in sample: 901
  - ✓ All included: True


In [29]:
# Save for reference
df_p80.to_csv('p80_companies.csv', index=False)

# API-Based Extraction of NBIM Voting Records

In [57]:
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NBIM_API_KEY')
BASE_URL = os.getenv('NBIM_BASE_URL')

In [58]:
import requests
import json

# =============================================================================
# API CONFIGURATION
# =============================================================================

API_KEY = "bgvpKNCRTt9QyhHQMdXc86uhmSCKYDoL6BsyFQKC"
BASE_URL = "https://vd.a.nbim.no"

headers = {
    'x-api-key': API_KEY
}

# =============================================================================
# EXPLORATION: TEST ENDPOINTS
# =============================================================================

# Test 1: Search by company name
print("="*60)
print("TEST 1: Query by company name (Microsoft)")
print("="*60)

company_name = "microsoft corporation"
response = requests.get(
    f"{BASE_URL}/v1/query/company/{company_name}",
    headers=headers
)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print(f"\nResponse structure:")
    print(json.dumps(data, indent=2)[:1000])  # First 1000 chars
    
    # Save full response
    with open('api_response_company.json', 'w') as f:
        json.dump(data, f, indent=2)
    print("\n✓ Full response saved to 'api_response_company.json'")
else:
    print(f"Error: {response.text}")


TEST 1: Query by company name (Microsoft)
Status code: 200

Response structure:
{
  "companies": [
    {
      "Ticker": "MSFT",
      "country": "United States",
      "id": 142533,
      "isin": "US5949181045",
      "meetings": [
        {
          "meetingDate": "2025-12-05 00:00:00",
          "meetingId": 2017007,
          "meetingType": "Annual"
        },
        {
          "meetingDate": "2024-12-10 00:00:00",
          "meetingId": 1906254,
          "meetingType": "Annual"
        },
        {
          "meetingDate": "2023-12-07 00:00:00",
          "meetingId": 1798481,
          "meetingType": "Annual"
        },
        {
          "meetingDate": "2022-12-13 00:00:00",
          "meetingId": 1694381,
          "meetingType": "Annual"
        },
        {
          "meetingDate": "2021-11-30 00:00:00",
          "meetingId": 1584741,
          "meetingType": "Annual"
        },
        {
          "meetingDate": "2020-12-02 00:00:00",
          "meetingId": 1479998,
  

In [26]:
# Get the first meeting ID from Microsoft's meetings
meeting_id = 2017007  # Most recent Microsoft meeting (2025-12-05)

print(f"TEST 2: Query meeting details (ID: {meeting_id})")
print("="*60)

# Make API request to get meeting details
response = requests.get(
    f"{BASE_URL}/v1/query/meeting/{meeting_id}",
    headers=headers
)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    
    # Print structure to console
    print(f"\nResponse structure:")
    print(json.dumps(data, indent=2))
    
else:
    print(f"Error: {response.text}")

TEST 2: Query meeting details (ID: 2017007)
Status code: 200

Response structure:
{
  "meeting": {
    "companyId": 142533,
    "companyName": "Microsoft Corporation",
    "companyTicker": "MSFT",
    "isin": "US5949181045",
    "meetingDate": "2025-12-05 00:00:00",
    "meetingId": 2017007,
    "meetingType": "Annual",
    "meetingVotes": [
      {
        "itemOnAgendaId": 17732594,
        "managementRec": "For",
        "meetingId": 2017007,
        "proponent": "Management",
        "proposalNumber": "1a",
        "proposalSequence": "1",
        "proposalText": "Elect Director Reid G. Hoffman",
        "voteInstruction": "For",
        "voterRationale": null
      },
      {
        "itemOnAgendaId": 17732595,
        "managementRec": "For",
        "meetingId": 2017007,
        "proponent": "Management",
        "proposalNumber": "1b",
        "proposalSequence": "2",
        "proposalText": "Elect Director Hugh F. Johnston",
        "voteInstruction": "For",
        "voterRatio

In [51]:
# LOAD P80 COMPANIES FOR PROCESSING
DATA_PATH = "../datasets/"
df_p80 =pd.read_csv(DATA_PATH +'p80_companies.csv')
df_p80.sample(3)

,Industry,Region,Country,Name,Market_Value_USD,Voting,Ownership,Portfolio_Weight,Environmental,Social,Governance,Climate_change,ESG_any
509,Consumer Staples,North America,United States,sprouts farmers market inc,143113350,1.13,1.13,0.011130,1,1,1,0,1
1742,Technology,Asia,Taiwan,phison electronics corp,82621884,2.47,2.47,0.006426,0,0,0,0,0
1536,Real Estate,North America,United States,lineage inc,680636054,5.09,5.09,0.052933,1,1,1,0,1


In [49]:
# Sort by Market Value descending
df_p80 = df_p80.sort_values('Market_Value_USD', ascending=False)

In [48]:
df_p80.head(8)

,Industry,Region,Country,Name,Market_Value_USD,Voting,Ownership,Portfolio_Weight,Environmental,Social,Governance,Climate_change,ESG_any
1598,Technology,North America,United States,apple inc,46210392003,1.22,1.22,3.593782,0,1,1,0,1
1709,Technology,North America,United States,microsoft corp,43758827987,1.40,1.40,3.403124,0,1,1,0,1
1731,Technology,North America,United States,nvidia corp,42973911250,1.31,1.31,3.342081,1,0,1,1,1
1593,Technology,North America,United States,alphabet inc,29271691564,0.88,1.26,2.276459,0,1,1,0,1
121,Consumer Discretionary,North America,United States,amazon.com inc,26979313029,1.17,1.17,2.098181,1,1,1,1,1
1706,Technology,North America,United States,meta platforms inc,19750530468,0.60,1.34,1.535999,1,1,1,1,1
1616,Technology,North America,United States,broadcom inc,16712325272,1.54,1.54,1.299717,0,0,1,0,1
1776,Technology,Asia,Taiwan,taiwan semiconductor manufacturing co ltd,15368120434,1.80,1.80,1.195179,0,0,0,0,0


In [37]:
import time
from datetime import datetime
import os

In [53]:
#just for a test
df_p80 = df_p80.head(50) 

In [40]:
print(f"Total companies to process: {len(df_p80)}")

# CHECKPOINT: Check if we have partial progress
checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

# Track progress
start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100  # Save every 100 companies

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    
    # Skip if already processed
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        # Step 1: Get company info and meetings
        response = requests.get(
            f"{BASE_URL}/v1/query/company/{company_name}",
            headers=headers,
            timeout=10
        )
        
        if response.status_code != 200:
            errors.append({'company': company_name, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        # Stats for this company
        total_disagreements = 0
        
        # Step 2: Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Get meeting details
            time.sleep(0.1)  # Rate limiting
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# Final save
df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 50
Progress: 50/50 (100.0%) - Est. 0.0 min remaining

EXTRACTION COMPLETE
Total companies processed: 50
Total errors: 17
Total disagreements found: 315

Companies with disagreements: 25
Companies with zero disagreements: 8


In [47]:
# View sample of disagreements
print("SAMPLE DISAGREEMENTS:")
print(df_disagreements.head(10))

print("\n" + "="*60)

# View which companies had errors
print("ERRORS:")
print(df_errors)

print("\n" + "="*60)

# View statistics per company
print("COMPANY STATS:")
print(df_stats.head(10))

SAMPLE DISAGREEMENTS:
  company_name ticker        country         meeting_date  meeting_id  \
0    apple inc   AAPL  United States  2024-02-28 00:00:00     1815577   
1    apple inc   AAPL  United States  2024-02-28 00:00:00     1815577   
2    apple inc   AAPL  United States  2023-03-10 00:00:00     1709502   
3    apple inc   AAPL  United States  2023-03-10 00:00:00     1709502   
4    apple inc   AAPL  United States  2022-03-04 00:00:00     1601071   
5    apple inc   AAPL  United States  2022-03-04 00:00:00     1601071   
6    apple inc   AAPL  United States  2022-03-04 00:00:00     1601071   
7    apple inc   AAPL  United States  2022-03-04 00:00:00     1601071   
8    apple inc   AAPL  United States  2022-03-04 00:00:00     1601071   
9    apple inc   AAPL  United States  2020-02-26 00:00:00     1388634   

  meeting_type proposal_number  \
0       Annual               6   
1       Annual               7   
2       Annual               3   
3       Annual               8   
4   

In [54]:
print(f"Total companies to process: {len(df_p80)}")

# CHECKPOINT: Check if we have partial progress
checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

# Track progress
start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    # Skip if already processed
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        # Step 1: Try by company name
        response = requests.get(
            f"{BASE_URL}/v1/query/company/{company_name}",
            headers=headers,
            timeout=10
        )
        
        # Check if first attempt failed
        if response.status_code != 200 or 'companies' not in response.json() or len(response.json()['companies']) == 0:
            # FALLBACK: Try by ticker
            if ticker:
                print(f"  → Retrying '{company_name}' with ticker '{ticker}'")
                time.sleep(0.1)
                response = requests.get(
                    f"{BASE_URL}/v1/query/ticker/{ticker}",
                    headers=headers,
                    timeout=10
                )
        
        # If still failed, log error and continue
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        # Stats for this company
        total_disagreements = 0
        
        # Step 2: Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Get meeting details
            time.sleep(0.1)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# Final save
df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 50
Progress: 50/50 (100.0%) - Est. 0.0 min remaining

EXTRACTION COMPLETE
Total companies processed: 50
Total errors: 7
Total disagreements found: 172

Companies with disagreements: 32
Companies with zero disagreements: 11


In [59]:
# Check which companies had errors
print("REMAINING ERRORS:")
print(df_errors[['company', 'ticker', 'error']])

print("\n" + "="*60)

# Check companies with most disagreements
print("TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_stats.nlargest(10, 'total_disagreements')[['company_name', 'total_disagreements']])

REMAINING ERRORS:
                          company ticker             error
0          agnico eagle mines ltd         No data returned
1  air products and chemicals inc         No data returned
2                 alamos gold inc         No data returned
3                   bhp group ltd         No data returned
4      cf industries holdings inc         No data returned
5    ganfeng lithium group co ltd         No data returned
6          grupo mexico sab de cv         No data returned

TOP 10 COMPANIES BY DISAGREEMENTS:
             company_name  total_disagreements
40          icl group ltd                   32
24             ecolab inc                   14
25       empresas cmpc sa                   14
22                dow inc                   10
23    eastman chemical co                   10
19   cleveland-cliffs inc                    9
36            givaudan sa                    8
3          albemarle corp                    7
26  ems-chemie holding ag                    7
35  

In [60]:
df_disagreements.head(3)

,company_name,ticker,country,meeting_date,meeting_id,meeting_type,proposal_number,proposal_text,proponent,management_rec,nbim_vote,position_paper,rationale_text
0,acerinox sa,ACX,Spain,2020-10-21 00:00:00,1471725,Annual,9,"Authorize Issuance of Convertible Bonds, Deben...",Management,For,Against,Position paper: Shareholder rights in equity i...,Existing shareholders should have the right to...
1,air liquide sa,AI,France,2021-05-04 00:00:00,1504230,Annual/Special,8,Elect Bertrand Dumazy as Director,Management,For,Against,Position paper: Time commitment of board members,Board members should devote sufficient time to...
2,albemarle corp,ALB,United States,2025-05-06 00:00:00,1946039,Annual,1c,"Elect Director J. Kent Masters, Jr.",Management,For,Against,Position paper: Separation of chairperson and CEO,The board should exercise objective judgement ...


In [70]:
df_disagreements[df_disagreements['company_name'].str.contains('gerdau')]

,company_name,ticker,country,meeting_date,meeting_id,meeting_type,proposal_number,proposal_text,proponent,management_rec,nbim_vote,position_paper,rationale_text
112,gerdau sa,GGBR4,Brazil,2025-04-10 00:00:00,1939048,Annual,1,"As a Preferred Shareholder, Would You like to ...",Management,None,Abstain,,
113,gerdau sa,GGBR4,Brazil,2025-04-10 00:00:00,1939048,Annual,2,Elect Denisio Augusto Liberato Delfino as Fisc...,Shareholder,None,For,,
114,gerdau sa,GGBR4,Brazil,2025-04-10 00:00:00,1939048,Annual,3,In Case Neither Class of Shares Reaches the Mi...,Management,None,For,,
115,gerdau sa,GGBR4,Brazil,2024-04-16 00:00:00,1835336,Annual,1,"As a Preferred Shareholder, Would You like to ...",Management,None,For,,
116,gerdau sa,GGBR4,Brazil,2024-04-16 00:00:00,1835336,Annual,2,Elect Claudio Antonio Goncalves as Director Ap...,Shareholder,None,For,,
117,gerdau sa,GGBR4,Brazil,2024-04-16 00:00:00,1835336,Annual,3,In Case Neither Class of Shares Reaches the Mi...,Management,None,For,,
118,gerdau sa,GGBR4,Brazil,2024-04-16 00:00:00,1835336,Annual,4,Elect Denisio Augusto Liberato Delfino as Fisc...,Shareholder,None,For,,


In [71]:
import re

In [74]:
df_p80[df_p80['Name'].str.contains('alamos')]

,Industry,Region,Country,Name,Market_Value_USD,Voting,Ownership,Portfolio_Weight,Environmental,Social,Governance,Climate_change,ESG_any
5,Basic Materials,North America,Canada,alamos gold inc,100771261,1.3,1.3,0.007837,0,0,0,0,0


In [ ]:
# Same dataframe will be used
df_p80.shape

(50, 13)

In [79]:
#Erasing previous checkpoints for trying a new approach to correct mistakes

# Delete old checkpoints
files_to_delete = ['checkpoint_progress.csv', 'checkpoint_disagreements.csv', 'checkpoint_stats.csv']

for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)
        print(f"✓ Deleted {file}")

print("\nCheckpoints cleared")


Checkpoints cleared


In [80]:
def clean_company_name(name):
    if pd.isna(name):
        return ""
    # Remove punctuation and parentheses
    name = re.sub(r'[.,()&/]', '', name)
    # Replace multiple spaces with single space
    name = ' '.join(name.split())
    return name.strip()

print(f"Total companies to process: {len(df_p80)}")

# CHECKPOINT: Check if we have partial progress
checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

# Track progress
start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    # Skip if already processed
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        # STRATEGY: Try ticker first (if available), then name
        response = None
        
        # Try 1: Search by ticker (more reliable)
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                # Ticker worked
                pass
            else:
                # Ticker failed, try name
                response = None
        
        # Try 2: Search by company name (if ticker failed or not available)
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        # If both failed, log error and continue
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        # Stats for this company
        total_disagreements = 0
        
        # Step 2: Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Get meeting details
            time.sleep(0.1)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # SKIP if management_rec is None or empty
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec):
                    continue
                
                # SKIP if nbim_vote is None or empty
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote):
                    continue
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# Final save
df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 50
Progress: 50/50 (100.0%) - Est. 0.0 min remaining

EXTRACTION COMPLETE
Total companies processed: 50
Total errors: 7
Total disagreements found: 172

Companies with disagreements: 32
Companies with zero disagreements: 11


In [81]:
# Check which companies had errors
print("REMAINING ERRORS:")
print(df_errors[['company', 'ticker', 'error']])

print("\n" + "="*60)

# Check companies with most disagreements
print("TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_stats.nlargest(10, 'total_disagreements')[['company_name', 'total_disagreements']])

REMAINING ERRORS:
                          company ticker             error
0          agnico eagle mines ltd         No data returned
1  air products and chemicals inc         No data returned
2                 alamos gold inc         No data returned
3                   bhp group ltd         No data returned
4      cf industries holdings inc         No data returned
5    ganfeng lithium group co ltd         No data returned
6          grupo mexico sab de cv         No data returned

TOP 10 COMPANIES BY DISAGREEMENTS:
             company_name  total_disagreements
40          icl group ltd                   32
24             ecolab inc                   14
25       empresas cmpc sa                   14
22                dow inc                   10
23    eastman chemical co                   10
19   cleveland-cliffs inc                    9
36            givaudan sa                    8
3          albemarle corp                    7
26  ems-chemie holding ag                    7
35  

In [86]:
# Check ICL disagreements - verify None filtering worked
icl_disagreements = df_disagreements[df_disagreements['company_name'].str.contains('icl', case=False)]

print(f"Total ICL disagreements: {len(icl_disagreements)}")
print("\nSample of ICL disagreements:")
print(icl_disagreements[['proposal_number', 'management_rec', 'nbim_vote']].head(10))

print("\n" + "="*60)
print("Checking for None values:")
print(f"Proposals with None in management_rec: {icl_disagreements['management_rec'].isna().sum()}")
print(f"Proposals with None as string: {(icl_disagreements['management_rec'] == 'None').sum()}")

Total ICL disagreements: 32

Sample of ICL disagreements:
    proposal_number management_rec nbim_vote
137               A           None   Against
138              B1           None   Against
139              B2           None   Against
140              B3           None   Against
141               A           None   Against
142              B1           None   Against
143              B2           None   Against
144              B3           None   Against
145               A           None   Against
146              B1           None   Against

Checking for None values:
Proposals with None in management_rec: 0
Proposals with None as string: 32


In [87]:
for file in ['checkpoint_progress.csv', 'checkpoint_disagreements.csv', 'checkpoint_stats.csv']:
    if os.path.exists(file):
        os.remove(file)

In [89]:
def clean_company_name(name):
    if pd.isna(name):
        return ""
    # Remove punctuation and parentheses
    name = re.sub(r'[.,()&/]', '', name)
    # Replace multiple spaces with single space
    name = ' '.join(name.split())
    return name.strip()

print(f"Total companies to process: {len(df_p80)}")

# CHECKPOINT: Check if we have partial progress
checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

# Track progress
start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    # Skip if already processed
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        # STRATEGY: Try ticker first (if available), then name
        response = None
        
        # Try 1: Search by ticker (more reliable)
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                # Ticker worked
                pass
            else:
                # Ticker failed, try name
                response = None
        
        # Try 2: Search by company name (if ticker failed or not available)
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        # If both failed, log error and continue
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        # Stats for this company
        total_disagreements = 0
        
        # Step 2: Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Get meeting details
            time.sleep(0.1)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # SKIP if management_rec is None, empty, or string "None"
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec) or str(mgmt_rec) == 'None':
                    continue
                
                # SKIP if nbim_vote is None, empty, or string "None"
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote) or str(nbim_vote) == 'None':
                    continue
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# Final save
df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 50
Progress: 50/50 (100.0%) - Est. 0.0 min remaining

EXTRACTION COMPLETE
Total companies processed: 50
Total errors: 7
Total disagreements found: 129

Companies with disagreements: 28
Companies with zero disagreements: 15


In [90]:
# Check which companies had errors
print("REMAINING ERRORS:")
print(df_errors[['company', 'ticker', 'error']])

print("\n" + "="*60)

# Check companies with most disagreements
print("TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_stats.nlargest(10, 'total_disagreements')[['company_name', 'total_disagreements']])

REMAINING ERRORS:
                          company ticker             error
0          agnico eagle mines ltd         No data returned
1  air products and chemicals inc         No data returned
2                 alamos gold inc         No data returned
3                   bhp group ltd         No data returned
4      cf industries holdings inc         No data returned
5    ganfeng lithium group co ltd         No data returned
6          grupo mexico sab de cv         No data returned

TOP 10 COMPANIES BY DISAGREEMENTS:
             company_name  total_disagreements
24             ecolab inc                   14
25       empresas cmpc sa                   14
22                dow inc                   10
23    eastman chemical co                   10
19   cleveland-cliffs inc                    9
36            givaudan sa                    8
3          albemarle corp                    7
26  ems-chemie holding ag                    7
38              holmen ab                    7
18  

In [91]:
#Restablishing the dataset to its original size and content
# LOAD P80 COMPANIES FOR PROCESSING
DATA_PATH = "../datasets/"
df_p80 =pd.read_csv(DATA_PATH +'p80_companies.csv')
df_p80.shape


(1925, 13)

In [92]:
#Erasing checkpoints for being able to analyze from scratch
for file in ['checkpoint_progress.csv', 'checkpoint_disagreements.csv', 'checkpoint_stats.csv']:
    if os.path.exists(file):
        os.remove(file)
        print(f"✓ Deleted {file}")

In [93]:
def clean_company_name(name):
    if pd.isna(name):
        return ""
    name = re.sub(r'[.,()&/]', '', name)
    name = ' '.join(name.split())
    return name.strip()

print(f"Total companies to process: {len(df_p80)}")

checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        response = None
        
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                pass
            else:
                response = None
        
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        total_disagreements = 0
        
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            time.sleep(0.1)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec) or str(mgmt_rec) == 'None':
                    continue
                
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote) or str(nbim_vote) == 'None':
                    continue
                
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 1925
Progress: 50/1925 (2.6%) - Est. 150.6 min remaining
Progress: 100/1925 (5.2%) - Est. 117.1 min remaining
Progress: 150/1925 (7.8%) - Est. 103.7 min remaining
Progress: 200/1925 (10.4%) - Est. 91.7 min remaining

✓ Checkpoint saved at 200 companies
Progress: 250/1925 (13.0%) - Est. 88.9 min remaining
Progress: 300/1925 (15.6%) - Est. 86.8 min remaining

✓ Checkpoint saved at 300 companies
Progress: 350/1925 (18.2%) - Est. 84.4 min remaining
Progress: 400/1925 (20.8%) - Est. 81.5 min remaining

✓ Checkpoint saved at 400 companies
Progress: 450/1925 (23.4%) - Est. 76.5 min remaining
Progress: 500/1925 (26.0%) - Est. 72.2 min remaining

✓ Checkpoint saved at 500 companies
Progress: 550/1925 (28.6%) - Est. 68.5 min remaining
Progress: 600/1925 (31.2%) - Est. 68.6 min remaining

✓ Checkpoint saved at 600 companies
Progress: 650/1925 (33.8%) - Est. 67.4 min remaining
Progress: 700/1925 (36.4%) - Est. 65.1 min remaining
Progress: 750/1925 (39.0%) - Est. 59.3 mi

In [94]:
# Check which companies had errors
print("REMAINING ERRORS:")
print(df_errors[['company', 'ticker', 'error']])

print("\n" + "="*60)

# Check companies with most disagreements
print("TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_stats.nlargest(10, 'total_disagreements')[['company_name', 'total_disagreements']])

REMAINING ERRORS:
                             company ticker             error
0             agnico eagle mines ltd         No data returned
1     air products and chemicals inc         No data returned
2                    alamos gold inc         No data returned
3                      bhp group ltd         No data returned
4         cf industries holdings inc         No data returned
...                              ...    ...               ...
1549      united utilities group plc               Status 429
1550         veolia environnement sa               Status 429
1551                     vistra corp               Status 429
1552           waste connections inc               Status 429
1553            waste management inc               Status 429

[1554 rows x 3 columns]

TOP 10 COMPANIES BY DISAGREEMENTS:
                 company_name  total_disagreements
218             volkswagen ag                   85
358    berkshire hathaway inc                   56
217                viven

In [95]:
# Error breakdown
print("ERROR BREAKDOWN:")
print(df_errors['error'].value_counts())

print("\n" + "="*60)
print(f"✓ Successfully processed: {len(df_stats)} empresas")
print(f"  - With disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"  - Zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")
print(f"\n✗ Failed: {len(df_errors)} empresas")
print(f"\nTotal disagreements found: {len(df_disagreements)}")

ERROR BREAKDOWN:
error
Status 429          1243
No data returned     311
Name: count, dtype: int64

✓ Successfully processed: 371 empresas
  - With disagreements: 243
  - Zero disagreements: 128

✗ Failed: 1554 empresas

Total disagreements found: 1805


In [96]:
# Error breakdown by type
print("ERROR BREAKDOWN:")
print(df_errors['error'].value_counts())

print("\n" + "="*60)

# Check at what point 429 errors started
df_errors['error_type'] = df_errors['error'].apply(lambda x: '429' if '429' in str(x) else 'Other')
print("\nError type distribution:")
print(df_errors['error_type'].value_counts())

ERROR BREAKDOWN:
error
Status 429          1243
No data returned     311
Name: count, dtype: int64


Error type distribution:
error_type
429      1243
Other     311
Name: count, dtype: int64


In [ ]:
def clean_company_name(name):
    if pd.isna(name):
        return ""
    name = re.sub(r'[.,()&/]', '', name)
    name = ' '.join(name.split())
    return name.strip()

print(f"Total companies to process: {len(df_p80)}")

checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        response = None
        
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                pass
            else:
                response = None
        
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        total_disagreements = 0
        
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            time.sleep(1.0)  # INCREASED from 0.1 to 1.0 seconds
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec) or str(mgmt_rec) == 'None':
                    continue
                
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote) or str(nbim_vote) == 'None':
                    continue
                
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

In [ ]:
# Verify checkpoint files exist
print("Checkpoint files exist:")
print(f"checkpoint_progress.csv: {os.path.exists('checkpoint_progress.csv')}")
print(f"checkpoint_disagreements.csv: {os.path.exists('checkpoint_disagreements.csv')}")
print(f"checkpoint_stats.csv: {os.path.exists('checkpoint_stats.csv')}")

# Verify how many companies already processed
if os.path.exists('checkpoint_progress.csv'):
    processed = pd.read_csv('checkpoint_progress.csv')
    print(f"\nCompanies already processed: {len(processed)}")

Checkpoint files exist:
checkpoint_progress.csv: True
checkpoint_disagreements.csv: True
checkpoint_stats.csv: True

Companies already processed: 600


Continue with the execution after revising the errors, checking the number of analyzed companies and slowing the time between API requests

In [98]:
def clean_company_name(name):
    if pd.isna(name):
        return ""
    # Remove punctuation and parentheses
    name = re.sub(r'[.,()&/]', '', name)
    # Replace multiple spaces with single space
    name = ' '.join(name.split())
    return name.strip()

print(f"Total companies to process: {len(df_p80)}")

checkpoint_file = 'checkpoint_progress.csv'
disagreements_file = 'checkpoint_disagreements.csv'
stats_file = 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies already done")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_p80.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    # Skip if already processed
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_p80) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_p80)} ({processed/len(df_p80)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        response = None
        
        # Try 1: Search by ticker (more reliable)
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                pass
            else:
                response = None
        
        # Try 2: Search by company name (if ticker failed or not available)
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        # If both failed, log error and continue
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        # Stats for this company
        total_disagreements = 0
        
        # Step 2: Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Rate limiting: 1 second delay between requests
            time.sleep(1.0)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # SKIP if management_rec is None, empty, or string "None"
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec) or str(mgmt_rec) == 'None':
                    continue
                
                # SKIP if nbim_vote is None, empty, or string "None"
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote) or str(nbim_vote) == 'None':
                    continue
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# Final save
df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

Total companies to process: 1925

✓ RESUMING from checkpoint: 600 companies already done
Progress: 650/1925 (33.8%) - Est. 0.8 min remaining
Progress: 700/1925 (36.4%) - Est. 1.5 min remaining
Progress: 750/1925 (39.0%) - Est. 1.9 min remaining
Progress: 800/1925 (41.6%) - Est. 2.2 min remaining
Progress: 850/1925 (44.2%) - Est. 2.4 min remaining
Progress: 900/1925 (46.8%) - Est. 2.8 min remaining
Progress: 950/1925 (49.4%) - Est. 2.9 min remaining
Progress: 1000/1925 (51.9%) - Est. 3.0 min remaining
Progress: 1050/1925 (54.5%) - Est. 3.0 min remaining
Progress: 1100/1925 (57.1%) - Est. 3.0 min remaining
Progress: 1150/1925 (59.7%) - Est. 3.0 min remaining
Progress: 1200/1925 (62.3%) - Est. 2.9 min remaining
Progress: 1250/1925 (64.9%) - Est. 2.8 min remaining
Progress: 1300/1925 (67.5%) - Est. 2.7 min remaining
Progress: 1350/1925 (70.1%) - Est. 2.5 min remaining
Progress: 1400/1925 (72.7%) - Est. 2.3 min remaining
Progress: 1450/1925 (75.3%) - Est. 2.1 min remaining
Progress: 1500/19

In [99]:
# Check which companies had errors
print("REMAINING ERRORS:")
print(df_errors[['company', 'ticker', 'error']])

print("\n" + "="*60)

# Check companies with most disagreements
print("TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_stats.nlargest(10, 'total_disagreements')[['company_name', 'total_disagreements']])

REMAINING ERRORS:
                               company ticker       error
0                           allianz se         Status 429
1                    allstate corp/the         Status 429
2      american financial group inc/oh         Status 429
3     american international group inc         Status 429
4             ameriprise financial inc         Status 429
...                                ...    ...         ...
1320        united utilities group plc         Status 429
1321           veolia environnement sa         Status 429
1322                       vistra corp         Status 429
1323             waste connections inc         Status 429
1324              waste management inc         Status 429

[1325 rows x 3 columns]

TOP 10 COMPANIES BY DISAGREEMENTS:
                 company_name  total_disagreements
218             volkswagen ag                   85
217                vivendi se                   46
159           mcdonald's corp                   41
169                op

In [100]:
# Check how many companies were actually processed in this run
print(f"Companies in checkpoint: {len(processed_companies)}")
print(f"Companies in df_stats: {len(df_stats)}")

# Check error breakdown
print("\nERROR BREAKDOWN:")
print(df_errors['error'].value_counts())

# Check if we processed any new companies
if os.path.exists('checkpoint_progress.csv'):
    checkpoint_before = 600  # We had 600 before
    checkpoint_now = len(pd.read_csv('checkpoint_progress.csv'))
    print(f"\nCompanies processed in this run: {checkpoint_now - checkpoint_before}")

Companies in checkpoint: 1925
Companies in df_stats: 329

ERROR BREAKDOWN:
error
Status 429    1325
Name: count, dtype: int64

Companies processed in this run: 0


In [102]:
# Check actual successful companies
print(f"Companies with actual data: {len(df_stats)}")
print(f"Total disagreements: {len(df_disagreements)}")

# Check companies with disagreements
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")

# See if we have enough data to work with
print(f"\nData coverage: {len(df_stats)/len(df_p80)*100:.1f}% of total sample")

Companies with actual data: 329
Total disagreements: 1614

Companies with disagreements: 215
Companies with zero disagreements: 114

Data coverage: 17.1% of total sample


In [103]:
# Check if we have the largest companies
print("COVERAGE ANALYSIS:")
print("="*60)

# Calculate market value coverage
total_market_value = df_p80['Market_Value_USD'].sum()
processed_names = df_stats['company_name'].tolist()
processed_market_value = df_p80[df_p80['Name'].isin(processed_names)]['Market_Value_USD'].sum()

print(f"Market value coverage: {processed_market_value/total_market_value*100:.1f}%")

# Check if we have top companies
df_p80['processed'] = df_p80['Name'].isin(processed_names)
print(f"\nTop 500 companies coverage: {df_p80.head(500)['processed'].sum()}/500")
print(f"Top 1000 companies coverage: {df_p80.head(1000)['processed'].sum()}/1000")

# Check ESG_any coverage
if 'ESG_any' in df_p80.columns:
    esg_companies = df_p80[df_p80['ESG_any'] == 1]['Name'].tolist()
    esg_processed = len([c for c in esg_companies if c in processed_names])
    print(f"\nESG_any companies coverage: {esg_processed}/{len(esg_companies)}")

# Check industry distribution
print("\nINDUSTRY COVERAGE (processed companies):")
processed_df = df_p80[df_p80['Name'].isin(processed_names)]
print(processed_df['Industry'].value_counts().head(10))

# Check disaggregments stats
print(f"\nDISAGREEMENTS:")
print(f"Total: {len(df_disagreements)}")
print(f"Average per company: {len(df_disagreements)/len(df_stats):.1f}")

COVERAGE ANALYSIS:
Market value coverage: 12.8%

Top 500 companies coverage: 273/500
Top 1000 companies coverage: 329/1000

ESG_any companies coverage: 208/901

INDUSTRY COVERAGE (processed companies):
Industry
Consumer Discretionary    146
Basic Materials            82
Consumer Staples           56
Energy                     35
Financials                 10
Name: count, dtype: int64

DISAGREEMENTS:
Total: 1614
Average per company: 4.9


In [105]:
#Save current data with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M')

# Save dataframes
df_disagreements.to_csv(f'disagreements_partial_{timestamp}.csv', index=False)
df_stats.to_csv(f'company_stats_partial_{timestamp}.csv', index=False)
df_errors.to_csv(f'errors_partial_{timestamp}.csv', index=False)

print(f"✓ Saved partial data with timestamp: {timestamp}")
print(f"  - {len(df_disagreements)} disagreements")
print(f"  - {len(df_stats)} companies")

✓ Saved partial data with timestamp: 20251209_1313
  - 1614 disagreements
  - 329 companies


In [106]:
# Merge with original df_p80 to get additional info
df_analysis = df_stats.merge(
    df_p80[['Name', 'Industry', 'Market_Value_USD', 'ESG_any']], 
    left_on='company_name', 
    right_on='Name', 
    how='left'
)

print("PRELIMINARY ANALYSIS (329 companies)")
print("="*60)

# Disagreement distribution
print("\n1. DISAGREEMENT DISTRIBUTION:")
print(f"Companies with disagreements: {(df_analysis['total_disagreements'] > 0).sum()} ({(df_analysis['total_disagreements'] > 0).sum()/len(df_analysis)*100:.1f}%)")
print(f"Companies with zero: {(df_analysis['total_disagreements'] == 0).sum()} ({(df_analysis['total_disagreements'] == 0).sum()/len(df_analysis)*100:.1f}%)")
print(f"\nTotal disagreements: {df_analysis['total_disagreements'].sum()}")
print(f"Average per company: {df_analysis['total_disagreements'].mean():.1f}")
print(f"Median: {df_analysis['total_disagreements'].median():.0f}")

# Top companies by disagreements
print("\n2. TOP 10 COMPANIES BY DISAGREEMENTS:")
print(df_analysis.nlargest(10, 'total_disagreements')[['company_name', 'Industry', 'total_disagreements']])

# Industry breakdown
print("\n3. DISAGREEMENTS BY INDUSTRY:")
industry_stats = df_analysis.groupby('Industry').agg({
    'total_disagreements': ['sum', 'mean', 'count']
}).round(2)
industry_stats.columns = ['Total', 'Avg_per_company', 'N_companies']
print(industry_stats.sort_values('Total', ascending=False))

# ESG_any comparison
print("\n4. ESG_ANY COMPARISON:")
if 'ESG_any' in df_analysis.columns:
    esg_comparison = df_analysis.groupby('ESG_any').agg({
        'total_disagreements': ['mean', 'sum', 'count']
    }).round(2)
    esg_comparison.columns = ['Avg_disagreements', 'Total', 'N_companies']
    print(esg_comparison)

PRELIMINARY ANALYSIS (329 companies)

1. DISAGREEMENT DISTRIBUTION:
Companies with disagreements: 215 (65.3%)
Companies with zero: 114 (34.7%)

Total disagreements: 1614
Average per company: 4.9
Median: 2

2. TOP 10 COMPANIES BY DISAGREEMENTS:
                 company_name                Industry  total_disagreements
218             volkswagen ag  Consumer Discretionary                   85
217                vivendi se  Consumer Discretionary                   46
159           mcdonald's corp  Consumer Discretionary                   41
169                opmobility  Consumer Discretionary                   41
302    petroleo brasileiro sa                  Energy                   40
188                    seb sa  Consumer Discretionary                   39
130  hermes international sca  Consumer Discretionary                   36
294          exxon mobil corp                  Energy                   36
103         christian dior se  Consumer Discretionary                   28
149   

In [107]:
# Check if we have tech companies
print("TECH SECTOR VERIFICATION:")
print("="*60)

# Check all unique industries
print("\nAll industries present:")
print(df_analysis['Industry'].value_counts())

# Search for tech-related companies by name
tech_keywords = ['microsoft', 'apple', 'nvidia', 'alphabet', 'google', 'meta', 'amazon', 'tesla']
df_analysis['is_tech_name'] = df_analysis['company_name'].str.lower().apply(
    lambda x: any(keyword in str(x) for keyword in tech_keywords)
)

print("\n\nTech companies found by name:")
tech_companies = df_analysis[df_analysis['is_tech_name']]
if len(tech_companies) > 0:
    print(tech_companies[['company_name', 'Industry', 'total_disagreements']])
else:
    print("❌ NO TECH COMPANIES FOUND IN 329 PROCESSED")

# Check original df_p80 to see tech distribution
print("\n" + "="*60)
print("TECH IN ORIGINAL SAMPLE:")
if 'Industry' in df_p80.columns:
    print(df_p80['Industry'].value_counts().head(15))

TECH SECTOR VERIFICATION:

All industries present:
Industry
Consumer Discretionary    146
Basic Materials            82
Consumer Staples           56
Energy                     35
Financials                 10
Name: count, dtype: int64


Tech companies found by name:
                        company_name         Industry  total_disagreements
74  triple flag precious metals corp  Basic Materials                    1
81      wheaton precious metals corp  Basic Materials                    2

TECH IN ORIGINAL SAMPLE:
Industry
Industrials               368
Financials                353
Consumer Discretionary    296
Technology                218
Health Care               167
Consumer Staples          119
Basic Materials           113
Real Estate               113
Utilities                  67
Energy                     58
Telecommunications         53
Name: count, dtype: int64


In [108]:
# Check where tech companies are in the ranking
print("TECH COMPANIES POSITION IN RANKING:")
print("="*60)

df_p80_ranked = df_p80.reset_index(drop=True)
df_p80_ranked['rank'] = df_p80_ranked.index + 1

tech_companies = df_p80_ranked[df_p80_ranked['Industry'] == 'Technology']

print(f"\nTech companies in sample: {len(tech_companies)}")
print(f"\nTech company positions (first 20):")
print(tech_companies.head(20)[['rank', 'Name', 'Market_Value_USD', 'Industry']])

print(f"\nLowest tech rank: {tech_companies['rank'].min()}")
print(f"Highest tech rank: {tech_companies['rank'].max()}")
print(f"Median tech rank: {tech_companies['rank'].median():.0f}")

# Companies we processed
print(f"\n\nCompanies processed: positions 1-329")
print(f"Tech starts at position: {tech_companies['rank'].min()}")
print(f"\n❌ We stopped BEFORE reaching tech companies!")

TECH COMPANIES POSITION IN RANKING:

Tech companies in sample: 218

Tech company positions (first 20):
      rank                           Name  Market_Value_USD    Industry
1587  1588                      adobe inc        2512307262  Technology
1588  1589     advanced micro devices inc        2837356134  Technology
1589  1590                 advantest corp         798576473  Technology
1590  1591        akamai technologies inc         474499181  Technology
1591  1592        alchip technologies ltd         209533519  Technology
1592  1593       allegro microsystems inc          26926405  Technology
1593  1594                   alphabet inc       29271691564  Technology
1594  1595            amadeus it group sa         813691936  Technology
1595  1596                  amphenol corp        1389817288  Technology
1596  1597             analog devices inc        1776064894  Technology
1597  1598                      ansys inc         622416691  Technology
1598  1599                      a

In [109]:
# 1. Save list of companies already processed (329)
processed_companies_list = df_stats['company_name'].tolist()
print(f"Companies already processed: {len(processed_companies_list)}")

# Save to file
import json
with open('companies_already_processed.json', 'w') as f:
    json.dump(processed_companies_list, f)
print("✓ Saved list of processed companies")

# 2. Re-order df_p80 correctly by market value
df_p80_sorted = df_p80.sort_values('Market_Value_USD', ascending=False).reset_index(drop=True)

# 3. Mark which companies are already done
df_p80_sorted['already_processed'] = df_p80_sorted['Name'].isin(processed_companies_list)

print("\nSTATUS SUMMARY:")
print(f"Total companies: {len(df_p80_sorted)}")
print(f"Already processed: {df_p80_sorted['already_processed'].sum()}")
print(f"Remaining to process: {(~df_p80_sorted['already_processed']).sum()}")

# 4. Show next companies to process (should be tech giants)
print("\nNEXT 20 COMPANIES TO PROCESS (largest remaining):")
next_to_process = df_p80_sorted[~df_p80_sorted['already_processed']].head(20)
print(next_to_process[['Name', 'Market_Value_USD', 'Industry']])

# 5. Save correctly sorted df
df_p80_sorted.to_csv('p80_companies_sorted_with_status.csv', index=False)
print("\n✓ Saved sorted df_p80 with processing status")

Companies already processed: 329
✓ Saved list of processed companies

STATUS SUMMARY:
Total companies: 1925
Already processed: 329
Remaining to process: 1596

NEXT 20 COMPANIES TO PROCESS (largest remaining):
                                         Name  Market_Value_USD  \
0                                   apple inc       46210392003   
1                              microsoft corp       43758827987   
2                                 nvidia corp       42973911250   
3                                alphabet inc       29271691564   
4                              amazon.com inc       26979313029   
5                          meta platforms inc       19750530468   
6                                broadcom inc       16712325272   
7   taiwan semiconductor manufacturing co ltd       15368120434   
8                                   tesla inc       14211259233   
9                      berkshire hathaway inc        9483089298   
10                             eli lilly & co        8

Reorganizing the project's folders

In [110]:

import shutil
from pathlib import Path

# Define paths
DATA_PATH = "../datasets/"
TEMP_PATH = "./temp/"
VOTING_DATA_PATH = "../datasets/voting_data/"

# Create directories if they don't exist
os.makedirs(TEMP_PATH, exist_ok=True)
os.makedirs(VOTING_DATA_PATH, exist_ok=True)

print("REORGANIZING FILES...")
print("="*60)

# 1. Move checkpoints to temp folder
checkpoint_files = ['checkpoint_progress.csv', 'checkpoint_disagreements.csv', 'checkpoint_stats.csv']
for file in checkpoint_files:
    if os.path.exists(file):
        shutil.move(file, TEMP_PATH + file)
        print(f"✓ Moved {file} → temp/")

# 2. Move final data files to datasets/voting_data/
data_files = [
    'disagreements_partial_20251209_1313.csv',
    'company_stats_partial_20251209_1313.csv', 
    'errors_partial_20251209_1313.csv'
]
for file in data_files:
    if os.path.exists(file):
        shutil.move(file, VOTING_DATA_PATH + file)
        print(f"✓ Moved {file} → datasets/voting_data/")

# 3. Move companies list to datasets
if os.path.exists('companies_already_processed.json'):
    shutil.move('companies_already_processed.json', DATA_PATH + 'companies_already_processed.json')
    print(f"✓ Moved companies_already_processed.json → datasets/")

# 4. Move sorted p80 to datasets
if os.path.exists('p80_companies_sorted_with_status.csv'):
    shutil.move('p80_companies_sorted_with_status.csv', DATA_PATH + 'p80_companies_sorted_with_status.csv')
    print(f"✓ Moved p80_companies_sorted_with_status.csv → datasets/")

print("\n" + "="*60)
print("REORGANIZATION COMPLETE")
print("="*60)
print("\nCurrent structure:")
print("  /notebooks/temp/ → Temporary checkpoints")
print("  /datasets/ → Master data + sorted p80")
print("  /datasets/voting_data/ → Partial results")

REORGANIZING FILES...
✓ Moved checkpoint_progress.csv → temp/
✓ Moved checkpoint_disagreements.csv → temp/
✓ Moved checkpoint_stats.csv → temp/
✓ Moved disagreements_partial_20251209_1313.csv → datasets/voting_data/
✓ Moved company_stats_partial_20251209_1313.csv → datasets/voting_data/
✓ Moved errors_partial_20251209_1313.csv → datasets/voting_data/
✓ Moved companies_already_processed.json → datasets/
✓ Moved p80_companies_sorted_with_status.csv → datasets/

REORGANIZATION COMPLETE

Current structure:
  /notebooks/temp/ → Temporary checkpoints
  /datasets/ → Master data + sorted p80
  /datasets/voting_data/ → Partial results


# Execute next

In [ ]:

# SETUP: Paths and imports
#It has all the imports as it was from scratch

import pandas as pd
import requests
import time
import json
import os
import re
from datetime import datetime
from dotenv import load_dotenv

# Load API credentials
load_dotenv()
API_KEY = os.getenv('NBIM_API_KEY')
BASE_URL = os.getenv('NBIM_BASE_URL')

headers = {'x-api-key': API_KEY}

# Define paths
DATA_PATH = "../datasets/"
TEMP_PATH = "./temp/"
VOTING_DATA_PATH = "../datasets/voting_data/"

# Create directories if needed
os.makedirs(TEMP_PATH, exist_ok=True)
os.makedirs(VOTING_DATA_PATH, exist_ok=True)

# =============================================================================
# LOAD DATA
# =============================================================================

# Load sorted df_p80 with processing status
df_p80 = pd.read_csv(DATA_PATH + 'p80_companies_sorted_with_status.csv')

# Filter only unprocessed companies
df_to_process = df_p80[~df_p80['already_processed']].copy()

print(f"Total companies in sample: {len(df_p80)}")
print(f"Already processed: {df_p80['already_processed'].sum()}")
print(f"To process today: {len(df_to_process)}")
print(f"\nStarting with: {df_to_process.iloc[0]['Name']}")

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def clean_company_name(name):
    if pd.isna(name):
        return ""
    name = re.sub(r'[.,()&/]', '', name)
    name = ' '.join(name.split())
    return name.strip()

# =============================================================================
# CHECKPOINT SETUP
# =============================================================================

checkpoint_file = TEMP_PATH + 'checkpoint_progress.csv'
disagreements_file = TEMP_PATH + 'checkpoint_disagreements.csv'
stats_file = TEMP_PATH + 'checkpoint_stats.csv'

if os.path.exists(checkpoint_file):
    processed_companies = pd.read_csv(checkpoint_file)['company_name'].tolist()
    all_disagreements = pd.read_csv(disagreements_file).to_dict('records')
    company_stats = pd.read_csv(stats_file).to_dict('records')
    print(f"\n✓ RESUMING from checkpoint: {len(processed_companies)} companies in this session")
else:
    processed_companies = []
    all_disagreements = []
    company_stats = []

# =============================================================================
# MAIN EXTRACTION LOOP
# =============================================================================

start_time = datetime.now()
processed = len(processed_companies)
errors = []
checkpoint_interval = 100

for idx, row in df_to_process.iterrows():
    company_name = row['Name']
    ticker = row.get('Ticker', '')
    
    # Skip if already processed in this session
    if company_name in processed_companies:
        continue
    
    processed += 1
    
    # Progress update every 50 companies
    if processed % 50 == 0:
        elapsed = (datetime.now() - start_time).seconds
        rate = processed / (elapsed + 1)
        remaining = (len(df_to_process) - processed) / rate / 60
        print(f"Progress: {processed}/{len(df_to_process)} ({processed/len(df_to_process)*100:.1f}%) - Est. {remaining:.1f} min remaining")
    
    try:
        response = None
        
        # Try 1: Search by ticker
        if ticker and ticker.strip():
            response = requests.get(
                f"{BASE_URL}/v1/query/ticker/{ticker}",
                headers=headers,
                timeout=10
            )
            
            if response.status_code == 200 and 'companies' in response.json() and len(response.json()['companies']) > 0:
                pass
            else:
                response = None
        
        # Try 2: Search by company name
        if response is None:
            clean_name = clean_company_name(company_name)
            response = requests.get(
                f"{BASE_URL}/v1/query/company/{clean_name}",
                headers=headers,
                timeout=10
            )
        
        # If both failed, log error and continue
        if response.status_code != 200:
            errors.append({'company': company_name, 'ticker': ticker, 'error': f'Status {response.status_code}'})
            processed_companies.append(company_name)
            continue
        
        company_data = response.json()
        
        # Check if companies array exists
        if 'companies' not in company_data or len(company_data['companies']) == 0:
            errors.append({'company': company_name, 'ticker': ticker, 'error': 'No data returned'})
            processed_companies.append(company_name)
            continue
        
        company_info = company_data['companies'][0]
        meetings = company_info.get('meetings', [])
        
        # Filter meetings from 2020 onwards
        meetings_2020_plus = [
            m for m in meetings 
            if datetime.strptime(m['meetingDate'], '%Y-%m-%d %H:%M:%S').year >= 2020
        ]
        
        total_disagreements = 0
        
        # Process each meeting
        for meeting in meetings_2020_plus:
            meeting_id = meeting['meetingId']
            meeting_date = meeting['meetingDate']
            
            # Rate limiting: 1 second delay
            time.sleep(1.0)
            
            meeting_response = requests.get(
                f"{BASE_URL}/v1/query/meeting/{meeting_id}",
                headers=headers,
                timeout=10
            )
            
            if meeting_response.status_code != 200:
                continue
            
            meeting_data = meeting_response.json()
            votes = meeting_data.get('meeting', {}).get('meetingVotes', [])
            
            # Identify disagreements
            for vote in votes:
                mgmt_rec = vote.get('managementRec', '')
                nbim_vote = vote.get('voteInstruction', '')
                
                # SKIP if management_rec is None, empty, or string "None"
                if mgmt_rec is None or mgmt_rec == '' or pd.isna(mgmt_rec) or str(mgmt_rec) == 'None':
                    continue
                
                # SKIP if nbim_vote is None, empty, or string "None"
                if nbim_vote is None or nbim_vote == '' or pd.isna(nbim_vote) or str(nbim_vote) == 'None':
                    continue
                
                # DISAGREEMENT: when different
                if mgmt_rec != nbim_vote:
                    total_disagreements += 1
                    
                    # Extract rationale info
                    rationale_info = vote.get('voterRationale', None)
                    position_paper = ''
                    rationale_text = ''
                    
                    if rationale_info and len(rationale_info) > 0:
                        position_paper = rationale_info[0].get('positionPaper', {}).get('name', '')
                        rationale_text = rationale_info[0].get('publicRationaleOutgoing', '')
                    
                    # Store disagreement
                    all_disagreements.append({
                        'company_name': company_name,
                        'ticker': company_info.get('Ticker', ''),
                        'country': company_info.get('country', ''),
                        'meeting_date': meeting_date,
                        'meeting_id': meeting_id,
                        'meeting_type': meeting.get('meetingType', ''),
                        'proposal_number': vote.get('proposalNumber', ''),
                        'proposal_text': vote.get('proposalText', ''),
                        'proponent': vote.get('proponent', ''),
                        'management_rec': mgmt_rec,
                        'nbim_vote': nbim_vote,
                        'position_paper': position_paper,
                        'rationale_text': rationale_text
                    })
        
        # Store company-level stats
        company_stats.append({
            'company_name': company_name,
            'ticker': company_info.get('Ticker', ''),
            'total_meetings_2020_plus': len(meetings_2020_plus),
            'total_disagreements': total_disagreements
        })
        
        processed_companies.append(company_name)
        
        # CHECKPOINT: Save progress every 100 companies
        if len(processed_companies) % checkpoint_interval == 0:
            pd.DataFrame({'company_name': processed_companies}).to_csv(checkpoint_file, index=False)
            pd.DataFrame(all_disagreements).to_csv(disagreements_file, index=False)
            pd.DataFrame(company_stats).to_csv(stats_file, index=False)
            print(f"\n✓ Checkpoint saved at {len(processed_companies)} companies")
        
    except Exception as e:
        errors.append({'company': company_name, 'ticker': ticker, 'error': str(e)})
        processed_companies.append(company_name)
        continue

# =============================================================================
# FINAL SAVE
# =============================================================================

df_disagreements = pd.DataFrame(all_disagreements)
df_stats = pd.DataFrame(company_stats)
df_errors = pd.DataFrame(errors)

# Save to voting_data folder with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
df_disagreements.to_csv(VOTING_DATA_PATH + f'disagreements_{timestamp}.csv', index=False)
df_stats.to_csv(VOTING_DATA_PATH + f'company_stats_{timestamp}.csv', index=False)
df_errors.to_csv(VOTING_DATA_PATH + f'errors_{timestamp}.csv', index=False)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed in this session: {processed}")
print(f"Total errors: {len(errors)}")
print(f"Total disagreements found: {len(df_disagreements)}")
print(f"\nCompanies with disagreements: {(df_stats['total_disagreements'] > 0).sum()}")
print(f"Companies with zero disagreements: {(df_stats['total_disagreements'] == 0).sum()}")
print(f"\nData saved to: {VOTING_DATA_PATH}")

In [111]:
# Load data we already have
df_disagreements = pd.read_csv(VOTING_DATA_PATH + 'disagreements_partial_20251209_1313.csv')

print(f"Total disagreements to classify: {len(df_disagreements)}")

# Check position papers
print("\nPosition papers present:")
print(df_disagreements['position_paper'].value_counts())

# Check proposal text samples
print("\n\nSample proposals:")
print(df_disagreements[['proposal_text', 'position_paper']].head(20))

Total disagreements to classify: 1614

Position papers present:
position_paper
Position paper: CEO remuneration                          301
Position paper: Separation of chairperson and CEO         248
Position paper: Board independence                        169
Position paper: Time commitment of board members          116
Position paper: Corporate sustainability reporting        106
Position paper: Shareholder rights in equity issuances     56
Position paper: Related-party transaction                  22
Position paper: Board diversity                            19
Position paper: Multiple share classes                     13
Name: count, dtype: int64


Sample proposals:
                                        proposal_text  \
0   Authorize Issuance of Convertible Bonds, Deben...   
1                   Elect Bertrand Dumazy as Director   
2                 Elect Director J. Kent Masters, Jr.   
3                          Adopt Simple Majority Vote   
4                 Elect Director